## Tool call

### What is Tool Call?

Tool call is one of the most useful, the most fundamental and the most important LLM ability in agent development. In tool calling, LLM generates a specific format output to invoke function from user's code. The function execution result can then be sent back to LLM for further processing. Tool call greatly extends the capability of LLM from text in, text out to text in, action out.

In this notebook, we will show you how to add tool call ability to your agent. We will create a weather agent which can provides dummy weather information. The agent will have a `get_weather` tool to retrieve weather information from a given city and given date. And also a `get_current_date` tool to get the current date.

In [1]:
from dataclasses import dataclass, field
import datetime
from typing import List, Optional
import base64
from io import BytesIO
from autogen_core.base import MessageContext, AgentId, AgentInstantiationContext
from autogen_core.components import DefaultTopicId, RoutedAgent, default_subscription, message_handler
from autogen_core.components.code_executor import CodeExecutor, extract_markdown_code_blocks
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
import tempfile
from autogen_core.components.tool_agent import ToolAgent, tool_agent_caller_loop
from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.components.models import OpenAIChatCompletionClient, FunctionExecutionResultMessage, FunctionExecutionResult
import random

from autogen_core.base import CancellationToken
from autogen_core.components.tools import FunctionTool, ToolSchema
from autogen_core.components._image import Image
from autogen_core.components._types import FunctionCall
from typing_extensions import Annotated
from pydantic import BaseModel
import json


C:\Users\xiaoyuz\AppData\Local\Temp\ipykernel_100116\147059402.py:19: FutureWarning: OpenAIChatCompletionClient moved to autogen_ext. Please import it from autogen_ext.modelsChatCompletionClient.
  from autogen_core.components.models import OpenAIChatCompletionClient, FunctionExecutionResultMessage, FunctionExecutionResult


# Load .env

In [2]:
%load_ext dotenv
%dotenv .env

## Create a Message data class
The `Message` class is a data contract class that holds a text content.

In [3]:
class Message(BaseModel):
    text: str
    source: Optional[str] = None # indicate which agent the message is from

## Define a tool

In AutoGen, to define a tool, simply create a python function and use `FunctionTool` class to wrap it. The `FunctionTool` class will automatically generate the schema for the function. The schema contains the description of the function, the input parameters and the output parameters, and is sent to LLM to help LLM understand the tool and how to call it.

In [4]:
async def get_weather(city: str, state: str, date: str) -> str: # is the return type matter here?
    print(f"Getting the weather for {city}, {state}")
    return f"The weather in {city}, {state} on {date} will be sunny."

# Create a get_weather tool
get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city and state on a given date.")
get_weather_tool.schema

{'name': 'get_weather',
 'description': 'Get the weather for a city and state on a given date.',
 'parameters': {'type': 'object',
  'properties': {'city': {'description': 'city',
    'title': 'City',
    'type': 'string'},
   'state': {'description': 'state', 'title': 'State', 'type': 'string'},
   'date': {'description': 'date', 'title': 'Date', 'type': 'string'}},
  'required': ['city', 'state', 'date']}}

## Create WeatherAssistant Agent

The `WeatherAssistant` has two tools:
- `get_weather`: Get weather information from a given city and a given date.
- `get_current_date`: Get the current date.

It has one message handler registered. The message handler will be invoked when it receives a `Message` from agent runtime. And it will invoke LLM to generate a response.

When making request to LLM, the schemas of both tools are also sent to LLM. LLM will determine which tool to call based on the input message. If a `tool_call` response is received, the message handler will invoke the corresponding tool and send the result back to LLM for further processing. If a normal response is received, the message handler will publish the response to the agent runtime without further processing.


In [8]:
@default_subscription
class WeatherAssistant(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("An agent with tools")
        self._system_messages: List[LLMMessage] = [SystemMessage("You are a helpful AI assistant.")]
        self._model_client = model_client
        self._get_weather_tool = FunctionTool(self.get_weather, description="Get the weather for a city and state on a given date.")
        self._get_date_tool = FunctionTool(self.get_date, description="Get the current date.")
        self._tools = [self._get_date_tool, self._get_weather_tool]

    async def get_weather(self, city: str, state: str, date: str) -> str: # is the return type matter here?
    # get the weather for a city and state
        print(f"Getting the weather for {city}, {state} on {date}")
        return f"The weather in {city}, {state} on {date} will be sunny."
    
    async def get_date(self) -> str:
        today = datetime.datetime.today()
        return f"today is {today.strftime('%Y-%m-%d')}"
    
    @message_handler
    async def handle_text_message(self, message: Message, ctx: MessageContext) -> None:
        # Create a session of messages.
        session: List[LLMMessage] = [UserMessage(content=message.text, source="user")]

        while True:
            completion = await self._model_client.create(
                session,
                tools=self._tools,
                cancellation_token=ctx.cancellation_token,
            )
            print(f"### {self.type}: \n{completion.content}")
            if not isinstance(completion.content, list):
                await self.publish_message(
                    Message(text=completion.content),
                    DefaultTopicId(),
                )
                
                break
            
            # run tool call
            for tool_call in completion.content:
                arguments = tool_call.arguments
                arguments = json.loads(tool_call.arguments)
                if tool_call.name == self._get_date_tool.name:
                    result = await self._get_date_tool.run_json(arguments, ctx.cancellation_token)
                    reply = Message(text=self._get_date_tool.return_value_as_string(result))
                    print(f"### {self.type}: \n{reply.text}")
                    await self.publish_message(
                        reply,
                        DefaultTopicId(),
                    )
                    session.append(AssistantMessage(content=[tool_call], source="assistant"))
                    session.append(FunctionExecutionResultMessage(content=[FunctionExecutionResult(content=reply.text, call_id=tool_call.id)]))
                elif tool_call.name == self._get_weather_tool.name:
                    result = await self._get_weather_tool.run_json(arguments, ctx.cancellation_token)
                    reply = Message(text=self._get_weather_tool.return_value_as_string(result))
                    print(f"### {self.type}: \n{reply.text}")
                    await self.publish_message(
                        reply,
                        DefaultTopicId(),
                    )
                    session.append(AssistantMessage(content=[tool_call], source="assistant"))
                    session.append(FunctionExecutionResultMessage(content=[FunctionExecutionResult(content=reply.text, call_id=tool_call.id)]))


## Add assistant agent to the runtime

In [9]:
# Create an local embedded runtime.
import os;
runtime = SingleThreadedAgentRuntime()
await WeatherAssistant.register(
        runtime,
        "assistant",
        lambda: WeatherAssistant(
            OpenAIChatCompletionClient(
                model="gpt-4o-mini",
                # get api key from env:OPENAI_API_KEY
                api_key=os.environ.get("OPENAI_API_KEY"),
            ),
        ),
    )

AgentType(type='assistant')

## Send a message to agent

In [10]:
# Start the runtime and publish a message to the assistant.
runtime.start()
await runtime.publish_message(
    Message(text = "Hello, I am Geeno, what's the weather in New York today"), DefaultTopicId()
)

await runtime.stop_when_idle()

### assistant: 
[FunctionCall(id='call_7UclTIJNK840BJdUp1rIGk9d', arguments='{}', name='get_date')]
### assistant: 
today is 2024-10-25
### assistant: 
[FunctionCall(id='call_ON1vC83ybFG0LHVu1mTFt8CP', arguments='{"city":"New York","state":"NY","date":"2024-10-25"}', name='get_weather')]
Getting the weather for New York, NY on 2024-10-25
### assistant: 
The weather in New York, NY on 2024-10-25 will be sunny.
### assistant: 
Hello Geeno! The weather in New York today (October 25, 2024) is sunny.


## Conclusion

Congratulations! You have successfully created a weather assistant agent which can query weather information for a given city and date.

## What we have learned
- How to create a tool.
- How to equip the agent with a tool.
- How to process tool call in the message handler.